In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import urllib
import numpy as np
import pickle

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks/scripts")

Get Scripts


In [ ]:
url = "https://imsdb.com/all-scripts.html"
request = urllib.request.Request(url)
result = urllib.request.urlopen(request)

list_soup = BeautifulSoup(result.read(), 'html.parser')
links = list_soup.find_all('a', href = True)

movie_urls = []
movie_titles = []
for movie in links:
  if movie["href"][-11:] == "Script.html":
    movie_urls.append( "https://imsdb.com/scripts/" + movie['href'][15:-12].replace(" ", "-") + ".html")
    movie_titles.append(movie['href'][15:-12].replace(" ", "-"))

movie_urls = np.array(movie_urls)
movie_titles = np.array(movie_titles)

In [ ]:
random_idx = np.random.choice(len(movie_urls), 100, replace = False)
random_movies_urls = movie_urls[random_idx]
random_movies_titles = movie_titles[random_idx]

In [ ]:
scripts = []
for url in random_movies_urls:
  try:
    request = urllib.request.Request(url)
    result = urllib.request.urlopen(request)

    movie_soup = BeautifulSoup(result.read(), 'html.parser')
    source = movie_soup.find_all("td", class_ = "scrtext")[0]
    if source.pre is not None:
      scripts.append(source.pre.get_text())
  except:
    print("Failed to parse script")

Failed to parse script
Failed to parse script
Failed to parse script


In [ ]:
for title, script in zip(random_movies_titles, scripts):
  with open(title + ".txt", "w") as script_file:
    if (len(script) > 100):
      script_file.write(script)

Get Images for Movies

In [ ]:
# https://imsdb.com/Movie%20Scripts/17%20Again%20Script.html
titles = []
urls = []
for file in os.listdir():
  if file[-4:] == ".txt":
    titles.append(file[:-4].replace("-", " "))
    movie_name = file[:-4].replace("-", "%20")
    url = "https://imsdb.com/Movie%20Scripts/"+ movie_name + "%20Script.html"
    urls.append(url)

image_links = {}
for url, title in zip(urls, titles):
  request = urllib.request.Request(url)
  result = urllib.request.urlopen(request)

  script_soup = BeautifulSoup(result.read(), 'html.parser')
  img = script_soup.find_all("img", class_="avimg")
  
  if len(img) > 0:
    image_links[title] = img[0]['src']
  else:
    image_links[title] = "/images/no-poster.gif"

In [ ]:
with open('image_dict.pickle', 'wb') as handle:
    pickle.dump(image_links, handle)

Get Release Dates and Genres

In [ ]:
correct_titles = []
urls = []
for file in os.listdir():
  if file[-4:] == ".txt":
    movie_name = file[:-4].replace("-", "%20")
    correct_titles.append(file[:-4].replace("-", " "))
    url = "https://imsdb.com/Movie%20Scripts/"+ movie_name + "%20Script.html"
    urls.append(url)

movie_tags = {}
for url, title in zip(urls, correct_titles):
  request = urllib.request.Request(url)
  result = urllib.request.urlopen(request)

  script_soup = BeautifulSoup(result.read(), 'html.parser')

  script_table = script_soup.find_all("table", class_ = "script-details")[0]

  genres = []
  release_date = ""
  for tag in script_table.find_all("a"):
    if tag.has_attr("href") and tag.get("href")[:7] == "/genre/":
      genres.append(tag.get("href")[7:])

  try:
    release_index = script_table.get_text().find("Movie Release Date : ") + len("Movie Release Date : ")
    release_date = script_table.get_text()[release_index:].split("\n")[0]

    if release_date[-4:-2] != "20" and release_date[-4:-2] != "19":
      release_date = "Release Date Not Found"
  except:
    release_date = "Release Date Not Found"
  
  movie_tags[title] = (genres, release_date)

print(movie_tags)

{'Honeydripper': (['Drama', 'Musical'], 'December 2007'), 'Colombiana': (['Action', 'Crime', 'Drama'], 'August 2011'), 'Addams Family, The': (['Comedy', 'Fantasy', 'Horror', 'Family'], 'November 1991'), 'Mighty Morphin Power Rangers: The Movie': (['Family', 'Action'], 'June 1995'), 'Above the Law': (['Action'], 'Release Date Not Found'), 'Law Abiding Citizen': (['Crime', 'Drama', 'Thriller'], 'October 2009'), 'Sweet Smell of Success': (['Drama', 'Film-Noir'], 'Release Date Not Found'), 'Omega Man': (['Drama', 'Sci-Fi', 'Thriller'], 'Release Date Not Found'), 'Star Wars: The Phantom Menace': (['Sci-Fi', 'Adventure', 'Action', 'Fantasy'], 'Release Date Not Found'), 'Semi Pro': ([''], 'Release Date Not Found'), 'Wrestler, The': (['Drama'], 'January 2009'), 'Black Rain': (['Action', 'Crime', 'Drama'], 'Release Date Not Found'), 'Lake Placid': (['Action', 'Comedy', 'Horror', 'Thriller'], 'July 1999'), 'Meet Joe Black': (['Fantasy', 'Mystery', 'Romance', 'Drama'], 'Release Date Not Found'), 

In [ ]:
with open('tag_dict.pickle', 'wb') as handle:
    pickle.dump(movie_tags, handle)

In [ ]:
with open('tag_dict.pickle', 'rb') as handle:
    tags = pickle.load(handle)
print(tags)

{'Honeydripper': (['Drama', 'Musical'], 'December 2007'), 'Colombiana': (['Action', 'Crime', 'Drama'], 'August 2011'), 'Addams Family, The': (['Comedy', 'Fantasy', 'Horror', 'Family'], 'November 1991'), 'Mighty Morphin Power Rangers: The Movie': (['Family', 'Action'], 'June 1995'), 'Above the Law': (['Action'], 'Release Date Not Found'), 'Law Abiding Citizen': (['Crime', 'Drama', 'Thriller'], 'October 2009'), 'Sweet Smell of Success': (['Drama', 'Film-Noir'], 'Release Date Not Found'), 'Omega Man': (['Drama', 'Sci-Fi', 'Thriller'], 'Release Date Not Found'), 'Star Wars: The Phantom Menace': (['Sci-Fi', 'Adventure', 'Action', 'Fantasy'], 'Release Date Not Found'), 'Semi Pro': ([''], 'Release Date Not Found'), 'Wrestler, The': (['Drama'], 'January 2009'), 'Black Rain': (['Action', 'Crime', 'Drama'], 'Release Date Not Found'), 'Lake Placid': (['Action', 'Comedy', 'Horror', 'Thriller'], 'July 1999'), 'Meet Joe Black': (['Fantasy', 'Mystery', 'Romance', 'Drama'], 'Release Date Not Found'), 